<a href="https://colab.research.google.com/github/EfeTascioglu/hackTheNorth2019/blob/master/QuakeTester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import xgboost as xgb
import math
import requests
import datetime as dt
from datetime import datetime
import sklearn as sk
from datetime import datetime

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
!wget https://github.com/efetascioglu/hackTheNorth2019/raw/master/query1.csv

In [0]:
df = pd.read_csv('query1.csv') # (Dataframe) 

In [0]:
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2019-09-14T21:10:52.737Z,54.5263,-35.1239,10.00,5.0,mb,NaN,70.0,9.637,1.20,us,us70005gh4,2019-09-14T21:29:19.040Z,Reykjanes Ridge,earthquake,9.4,1.9,0.025,493.0,reviewed,us,us
1,2019-09-14T16:42:07.458Z,-30.2566,-177.8124,10.00,5.1,mb,NaN,161.0,1.013,1.25,us,us70005ge6,2019-09-14T17:01:57.040Z,"110km S of Raoul Island, New Zealand",earthquake,13.4,1.9,0.106,29.0,reviewed,us,us
2,2019-09-14T16:21:28.537Z,-0.9714,128.5726,10.00,5.9,mww,NaN,42.0,2.110,0.78,us,us70005ge3,2019-09-14T18:23:50.032Z,"108km ENE of Laiwui, Indonesia",earthquake,6.6,1.8,0.061,26.0,reviewed,us,us
3,2019-09-14T14:40:52.996Z,-33.9419,-73.2487,10.00,4.7,mb,NaN,152.0,1.629,0.87,us,us70005gde,2019-09-14T15:42:21.040Z,"155km WSW of San Antonio, Chile",earthquake,6.3,1.9,0.080,47.0,reviewed,us,us
4,2019-09-14T13:25:55.075Z,53.9819,-165.9177,73.06,5.4,mww,NaN,49.0,0.176,1.11,us,us70005gcr,2019-09-15T01:36:03.274Z,"19km SSW of Akutan, Alaska",earthquake,6.5,3.5,0.048,41.0,reviewed,us,us


In [0]:
df = df.drop('id', axis=1)
df = df.drop('net', axis=1)
df = df.drop('magType', axis=1)
df = df.drop('nst', axis=1)
df = df.drop('type', axis=1)
df = df.drop('status', axis=1)
df = df.drop('locationSource', axis=1)
df = df.drop('magSource', axis=1)
df = df.drop('updated', axis=1)
df = df.drop('depthError', axis=1)
df = df.drop('magError', axis=1)


# df = df.drop('time', axis=1)

In [6]:
df.dropna()
df.head()

,time,latitude,longitude,depth,mag,gap,dmin,rms,place,horizontalError,magNst
0,2019-09-14T21:10:52.737Z,54.5263,-35.1239,10.00,5.0,70.0,9.637,1.20,Reykjanes Ridge,9.4,493.0
1,2019-09-14T16:42:07.458Z,-30.2566,-177.8124,10.00,5.1,161.0,1.013,1.25,"110km S of Raoul Island, New Zealand",13.4,29.0
2,2019-09-14T16:21:28.537Z,-0.9714,128.5726,10.00,5.9,42.0,2.110,0.78,"108km ENE of Laiwui, Indonesia",6.6,26.0
3,2019-09-14T14:40:52.996Z,-33.9419,-73.2487,10.00,4.7,152.0,1.629,0.87,"155km WSW of San Antonio, Chile",6.3,47.0
4,2019-09-14T13:25:55.075Z,53.9819,-165.9177,73.06,5.4,49.0,0.176,1.11,"19km SSW of Akutan, Alaska",6.5,41.0


In [25]:
from fastai.tabular.transform import add_datepart
add_datepart(df, 'time') # inplace
df.head()

,latitude,longitude,depth,mag,gap,dmin,rms,place,horizontalError,magNst,timeYear,timeMonth,timeWeek,timeDay,timeDayofweek,timeDayofyear,timeIs_month_end,timeIs_month_start,timeIs_quarter_end,timeIs_quarter_start,timeIs_year_end,timeIs_year_start,timeElapsed
0,54.5263,-35.1239,10.00,5.0,70.0,9.637,1.20,Reykjanes Ridge,9.4,493.0,2019,9,37,14,5,257,False,False,False,False,False,False,1568495452
1,-30.2566,-177.8124,10.00,5.1,161.0,1.013,1.25,"110km S of Raoul Island, New Zealand",13.4,29.0,2019,9,37,14,5,257,False,False,False,False,False,False,1568479327
2,-0.9714,128.5726,10.00,5.9,42.0,2.110,0.78,"108km ENE of Laiwui, Indonesia",6.6,26.0,2019,9,37,14,5,257,False,False,False,False,False,False,1568478088
3,-33.9419,-73.2487,10.00,4.7,152.0,1.629,0.87,"155km WSW of San Antonio, Chile",6.3,47.0,2019,9,37,14,5,257,False,False,False,False,False,False,1568472052
4,53.9819,-165.9177,73.06,5.4,49.0,0.176,1.11,"19km SSW of Akutan, Alaska",6.5,41.0,2019,9,37,14,5,257,False,False,False,False,False,False,1568467555


In [0]:
#y_train, y_test = pd.DataFrame(train.iloc[:, 8]), pd.DataFrame(test.iloc[:, 8])
#x_train = pd.DataFrame(train.iloc[:, :8] + train.iloc[:, :8])
#x_test = pd.DataFrame(test[['Latitude', 'Longitude']])

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101749 entries, 0 to 101748
Data columns (total 23 columns):
latitude                101749 non-null float64
longitude               101749 non-null float64
depth                   101749 non-null float64
mag                     101749 non-null float64
gap                     100196 non-null float64
dmin                    43846 non-null float64
rms                     97895 non-null float64
place                   101747 non-null object
horizontalError         36403 non-null float64
magNst                  81688 non-null float64
timeYear                101749 non-null int64
timeMonth               101749 non-null int64
timeWeek                101749 non-null int64
timeDay                 101749 non-null int64
timeDayofweek           101749 non-null int64
timeDayofyear           101749 non-null int64
timeIs_month_end        101749 non-null bool
timeIs_month_start      101749 non-null bool
timeIs_quarter_end      101749 non-null bool
tim

In [0]:
#y = pd.DataFrame(df.iloc[:, 4])
#x = pd.DataFrame((df.iloc[:, :4]) + (df.iloc[:, 5:8]) + (df.iloc[:, 9:]))
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


In [0]:
#df['time'] = pd.to_datetime(df['time'])

In [0]:
y = pd.DataFrame(df[["mag"]])
x = pd.DataFrame(df[["latitude", "longitude", "depth", "gap", "dmin", "rms", "horizontalError", "magNst"]])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [0]:
#y = pd.DataFrame(df.iloc[:, 4])
#x = pd.DataFrame((df.iloc[:, :4]) + (df.iloc[:, 5:]))
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


In [0]:
#x_train['time'] = pd.to_datetime(x_train['time'])#.dt.tz_convert('US/Eastern')
#x_test['time']  = pd.to_datetime(x_test['time'])#.dt.tz_convert('US/Eastern')

In [0]:
x_train.shape

(81399, 8)

In [0]:
#train_x, test, test = train_test_split(df, test_size = 0.2)

In [8]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81399 entries, 62473 to 85712
Data columns (total 8 columns):
latitude           81399 non-null float64
longitude          81399 non-null float64
depth              81399 non-null float64
gap                80155 non-null float64
dmin               35136 non-null float64
rms                78324 non-null float64
horizontalError    29132 non-null float64
magNst             65381 non-null float64
dtypes: float64(8)
memory usage: 5.6 MB


In [0]:
from sklearn import preprocessing

x = x_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_train = pd.DataFrame(x_scaled)

In [0]:
x = y_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
y_train = pd.DataFrame(x_scaled)

In [0]:
x = x_test.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_test = pd.DataFrame(x_scaled)

In [0]:
x = y_test.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
y_test = pd.DataFrame(x_scaled)

In [40]:
x_train.shape

(81399, 8)

In [0]:
#x_train['time'].nanosecond

In [0]:
d_train = xgb.DMatrix(data=x_train, label=y_train) # Everything (Train)
d_test = xgb.DMatrix(data = x_test, label=y_test)

Scikit Learn
XGBoost
MatPlotLib

In [48]:
params = {'booster': 'gblinear', 
          'objective': 'reg:linear',
          'colsample_bytree': 1,
          'learning_rate': 0.2,
          'max_depth': 4,
          'lambda': 3,
          'num_class': 7}


#xgb_model = xgb.train(params=params,
#                      dtrain=d_train,
#                      evals=[(d_train, 'train'), (d_test, 'eval')],
#                      num_boost_round=100,
#                      early_stopping_rounds=20,
#                      verbose_eval = True)


cv_train = xgb.cv(dtrain=d_train,
                   params=params,
                   nfold=10,
                   num_boost_round=200,
                   early_stopping_rounds=20,
                   shuffle = True,
                   verbose_eval = True)


[13:58:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBoostError: ignored

In [27]:
xgbData = xgb.XGBRegressor(colsample_bytree=0.8, subsample=0.5,
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.8, n_estimators=2000,
                             reg_alpha=0.1, reg_lambda=0.3, gamma=0.01, 
                             silent=1, random_state =7, nthread = -1)

xgb_model = xgb.train(params=xgbData,
                      dtrain=d_train,
                      evals=[(d_train, 'train'), (d_test, 'eval')],
                      num_boost_round=100,
                      early_stopping_rounds=20,
                      verbose_eval = True
                      )

TypeError: ignored

In [23]:
xgbData = xgb.XGBRegressor(colsample_bytree=0.8, subsample=0.5,
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.8, n_estimators=2000,
                             reg_alpha=0.1, reg_lambda=0.3, gamma=0.01, 
                             silent=1, random_state =7, nthread = -1)


xgbData.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.01,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1.8, missing=None, n_estimators=2000,
             n_jobs=1, nthread=-1, objective='reg:linear', random_state=7,
             reg_alpha=0.1, reg_lambda=0.3, scale_pos_weight=1, seed=None,
             silent=1, subsample=0.5, verbosity=1)

In [0]:
xgdmat=xgb.DMatrix(x_train,y_train)
our_params={'eta':0.1,'seed':0,'subsample':0.8,'colsample_bytree':0.8,'objective':'reg:linear','max_depth':4,'min_child_weight':1}
final_gb=xgb.train(our_params,xgdmat)
tesdmat=xgb.DMatrix(X_test)
y_pred=final_gb.predict(tesdmat)
print(y_pred)